In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

spark_session = SparkSession.builder\
        .master("spark://192.168.2.250:7077") \
        .appName("A3_AN_Spark") \
        .config("spark.dynamicAllocation.enabled", True) \
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True) \
        .config("spark.shuffle.service.enabled", False) \
        .config("spark.dynamicAllocation.executorIdleTimeout", "30s") \
        .config("spark.executor.cores", 4) \
        .config("spark.driver.port", 9999) \
        .config("spark.blockManager.port", 10005) \
        .getOrCreate()

# RDD_API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

## Question B.1 ##

df = spark_session.read.option("header", True).csv('hdfs://192.168.2.250:9000/parking-citations.csv')
df.show()

## Question B.2 ##

df.printSchema()

## Question B.3 ##

df.count()

## Question B.4 ##

numParts = df.rdd.getNumPartitions()
print(numParts)

## Question B.5 ##

df = df.drop("VIN", "Latitude", "Longitude")

## Question B.6 ##

df = df.withColumn("Fine amount", df["Fine amount"].cast("float"))
maxFine = df.agg(F.max(df["Fine amount"])).collect()
print("Highest fine amount: ", maxFine)

## Question B.7 ##

makeFreqs = df.groupBy("Make").count().sort("count", ascending=False).show()

## Question B.8 ##

COLORS = {
'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black',
'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze',
'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold',
'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory',
'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon',
'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver',
'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White', 'WH':'White',
'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'
}

newCol = StringType()
def colorMapper(color):
    return COLORS.get(color, color)

colorUDF = udf(colorMapper, newCol)
df = df.withColumn("Color", colorUDF(df["color long"]))
df.show()

## Question B.9 ##
